In [130]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [131]:
def find_last_valid_record(df):
    '''Find the first row where all columns are null, and return the row previous to that.'''

    last_record_idx = df.isnull().apply(lambda x: all(x), axis=1).idxmax() - 1
    
    return last_record_idx


In [191]:
fname = "corporate sponsors 2022.xlsx"
#df = pd.read_excel(fname, sheet_name = '2010')

colnames = [ 'Added/   Modified', 	'Company Name',	'Sponsorship level'	,
             '$ Amount', 'Actual \n$ Amount',	'Money/Prize Received',  'Prize/Donation details' ]


In [209]:
df = pd.read_excel(fname, sheet_name = '2010', parse_dates=False)

In [213]:
df.iloc[:,1].head(50)

0     2019-01-01 00:00:00
1     2019-01-01 00:00:00
2     2019-01-01 00:00:00
3     2019-01-01 00:00:00
4     2020-01-01 00:00:00
5     2020-01-01 00:00:00
6     2022-01-01 00:00:00
7     2022-01-01 00:00:00
8     2025-01-01 00:00:00
9     2025-01-01 00:00:00
10    2012-01-01 00:00:00
11    2026-01-01 00:00:00
12    2026-01-01 00:00:00
13    2026-01-01 00:00:00
14    2026-01-01 00:00:00
15    2026-01-01 00:00:00
16    2027-01-01 00:00:00
17    2028-01-01 00:00:00
18    1931-01-01 00:00:00
19    1931-01-01 00:00:00
20    1931-01-01 00:00:00
21    1931-01-01 00:00:00
22    1931-01-01 00:00:00
23    1931-01-01 00:00:00
24    1931-01-01 00:00:00
25    2001-02-01 00:00:00
26    2001-02-01 00:00:00
27    2002-02-01 00:00:00
28    2002-02-01 00:00:00
29    2002-02-01 00:00:00
30    2002-02-01 00:00:00
31    2003-02-01 00:00:00
32    2003-02-01 00:00:00
33    2014-02-01 00:00:00
34                    NaN
35                    NaN
36    2009-02-01 00:00:00
37    2014-02-01 00:00:00
38    2006-0

dtype('O')

In [192]:
def process_year_donations(main_fname: str, year:str, colnames: list):
    df = pd.read_excel(main_fname, sheet_name = year)

    # we deduce if the column headers are in the first or second row. If necessary, we correct the read of the file
    c = 0
    for i in df.columns[:20]:
        if 'Unnamed:' in str(i):
            c += 1
    if c > 8 :
        df = pd.read_excel(main_fname, sheet_name = year, header = 1)

    last_record_idx = find_last_valid_record(df)
    df = df.iloc[:last_record_idx]

    df = df[df.columns & colnames].copy()

    #df['Added/   Modified'] = pd.to_datetime(df['Added/   Modified'])

    if 'Prize/Donation details' in df.columns:
        df['Donation details'] = df['Prize/Donation details']
    else:
        df['Donation details'] = df['Sponsorship level']
    
    df = df.drop(columns = ['Sponsorship level'	,'Prize/Donation details'], errors = 'ignore')
    
    df = df.rename(columns = {'Actual \n$ Amount':'$ Amount'})

    df = df.reset_index(drop=True)

    return df



In [158]:
years = [str(i) for i in range(2010,2023,1)]

In [193]:
df_ = process_year_donations(fname, '2016', colnames)


In [194]:
df_.columns

Index(['Added/   Modified', 'Company Name', '$ Amount', 'Money/Prize Received',
       'Donation details'],
      dtype='object')

In [244]:
df_concat = process_year_donations(fname, years[0], colnames)[:0] # we just take the structure

for y in years:
    df_ = process_year_donations(fname, y, colnames)
    print(y , df_.shape , '\n')
    df_['Year'] = y 
    df_concat = pd.concat( [df_concat, df_])

df_concat = df_concat.reset_index(drop=True)



2010 (80, 5) 

2011 (86, 5) 

2012 (84, 5) 

2013 (93, 5) 

2014 (140, 5) 

2015 (149, 5) 

2016 (133, 5) 

2017 (103, 5) 

2018 (77, 5) 

2019 (56, 5) 

2020 (29, 5) 

2021 (33, 5) 

2022 (15, 5) 



In [245]:
df_concat.shape

(1078, 6)

In [246]:
df_concat

,Added/ Modified,Company Name,$ Amount,Money/Prize Received,Donation details,Year
0,2019-01-01 00:00:00,VIA,0,NaN,trip for 4 Montreal/Windsor $2600 value,2010
1,2019-01-01 00:00:00,Ultramar,500,yes,Silver - $500 in gas coupons,2010
2,2019-01-01 00:00:00,Fondation BBB,5000,yes,Platinum,2010
3,2019-01-01 00:00:00,Sleeman Unibroue Inc.,0,NaN,beer,2010
4,2020-01-01 00:00:00,Friends of Trouser Lake,500,NaN,Mike will look into ad,2010
...,...,...,...,...,...,...
1073,2022-02-23 00:00:00,Lorne Steinberg Wealth Management,2000.0,NaN,"$2,000 Silver Sponsorship",2022
1074,2022-02-23 00:00:00,"BMA - Brais, Malouin et Associés inc",2000.0,Yes,Corporate team,2022
1075,2021-12-18 00:00:00,Hemmingford Golf Club,0.0,NaN,NaN,2022
1076,NaN,Magie de l'air,0.0,NaN,NaN,2022


In [247]:
import re
cols = df_concat.columns.to_list()
new_cols=[]
for col in cols:
    
    col = re.sub(' +', ' ', col)
    col = col.replace(' ','_')
    col = col.replace('/','')
    col = col.replace('$','dollar')
    col = col.lower()
    new_cols.append(col)

df_concat.columns = new_cols

In [248]:
df_concat.added_modified = df_concat.added_modified.astype(str)

In [265]:
df_concat.added_modified.str[4:5].value_counts()

#df_concat.added_modified.str[7:8].value_counts()

-    976
      65
5      8
4      8
1      6
9      4
/      3
2      2
3      2
7      2
6      2
Name: added_modified, dtype: int64

In [268]:
date_format_flg = (df_concat.added_modified.str[4:5] == '-') & (df_concat.added_modified.str[7:8] == '-')

In [269]:
df_concat[date_format_flg]

## try to give the correct date format to added_modified. Older years seem to have only month/day 
## but most recent have also year, so a criteria has to be set to correctly parse 

,added_modified,company_name,dollar_amount,moneyprize_received,donation_details,year
0,2019-01-01 00:00:00,VIA,0,NaN,trip for 4 Montreal/Windsor $2600 value,2010
1,2019-01-01 00:00:00,Ultramar,500,yes,Silver - $500 in gas coupons,2010
2,2019-01-01 00:00:00,Fondation BBB,5000,yes,Platinum,2010
3,2019-01-01 00:00:00,Sleeman Unibroue Inc.,0,NaN,beer,2010
4,2020-01-01 00:00:00,Friends of Trouser Lake,500,NaN,Mike will look into ad,2010
...,...,...,...,...,...,...
1072,2022-06-15 00:00:00,Hardline Curling,0.0,NaN,Longtime sponsor,2022
1073,2022-02-23 00:00:00,Lorne Steinberg Wealth Management,2000.0,NaN,"$2,000 Silver Sponsorship",2022
1074,2022-02-23 00:00:00,"BMA - Brais, Malouin et Associés inc",2000.0,Yes,Corporate team,2022
1075,2021-12-18 00:00:00,Hemmingford Golf Club,0.0,NaN,NaN,2022
